In [3]:
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns

**HERE WE'RE GOING TO CHANGE CUSTOMERID TO A STRING VALUE INSTEAD OF A INTERGER AND THEN WE'RE SETTING THE DATAFRAME TO SALES_DATA**

In [4]:
data_type = {'_CustomerID': str }
sales_data = pd.read_csv('sales_data.csv', dtype=data_type, parse_dates=['OrderDate'])

In [5]:
sales_data.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000101,In-Store,WARE-UHY1004,12/31/2017,2018-05-31,6/14/2018,6/19/2018,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,12/31/2017,2018-05-31,6/22/2018,7/2/2018,USD,14,20,196,27,3,0.075,3939.6,3348.660
2,SO - 000103,Distributor,WARE-UHY1004,12/31/2017,2018-05-31,6/21/2018,7/1/2018,USD,21,16,213,16,1,0.050,1775.5,781.220
3,SO - 000104,Wholesale,WARE-NMK1003,12/31/2017,2018-05-31,6/2/2018,6/7/2018,USD,28,48,107,23,8,0.075,2324.9,1464.687
4,SO - 000105,Distributor,WARE-NMK1003,4/10/2018,2018-05-31,6/16/2018,6/26/2018,USD,22,49,111,26,8,0.100,1822.4,1476.144


In [6]:
sales_data.tail()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
7986,SO - 0008087,In-Store,WARE-MKL1006,9/26/2020,2020-12-30,1/7/2021,1/14/2021,USD,9,41,339,29,1,0.075,234.5,121.940
7987,SO - 0008088,Online,WARE-NMK1003,9/26/2020,2020-12-30,1/2/2021,1/4/2021,USD,14,29,202,3,6,0.050,3202.6,1921.560
7988,SO - 0008089,Online,WARE-UHY1004,9/26/2020,2020-12-30,1/23/2021,1/26/2021,USD,14,32,241,35,5,0.200,3825.7,2792.761
7989,SO - 0008090,Online,WARE-NMK1003,9/26/2020,2020-12-30,1/20/2021,1/25/2021,USD,20,42,112,36,8,0.100,1072.0,804.000
7990,SO - 0008091,In-Store,WARE-UHY1004,9/26/2020,2020-12-30,1/13/2021,1/19/2021,USD,6,41,237,43,5,0.075,2211.0,1370.820


**HERE WE ARE DOING A CALCULATION TO DETERMINE THE REVENUE COUNT**

In [7]:
df = sales_data.copy()
df['Revenue'] = (df['Unit Price'] - (df['Unit Price'] * df['Discount Applied']) - df['Unit Cost']) * df['Order Quantity']

In [8]:
df.dtypes

OrderNumber                 object
Sales Channel               object
WarehouseCode               object
ProcuredDate                object
OrderDate           datetime64[ns]
ShipDate                    object
DeliveryDate                object
CurrencyCode                object
_SalesTeamID                 int64
_CustomerID                 object
_StoreID                     int64
_ProductID                   int64
Order Quantity               int64
Discount Applied           float64
Unit Price                 float64
Unit Cost                  float64
Revenue                    float64
dtype: object

In [9]:
columns = ['OrderNumber', '_CustomerID', 'OrderDate', 'Revenue']
df_dataset = df[columns]

**HERE WE ARE GROUPING DATA BY CUSTOMER ID TO DETERMINE THE RECENCY**

In [10]:
today_date = pd.to_datetime('2021-01-01')

rfm_dataset = df_dataset.groupby('_CustomerID').agg({
    'OrderDate': lambda v: (today_date - v.max()).days,
    'OrderNumber': 'count',
    'Revenue': 'sum'
})

**HERE WE ARE RENAMING OUR COLUMNS TO RECENCY FREQUENCY AND MONETARY VALUE TO PROPERLY ANALYSE DATA IN THE FUTURE**

In [11]:
rfm_dataset.rename(
    columns= {
        'OrderDate': 'recency',
        'OrderNumber': 'frequency',
        'Revenue': 'monetary'
    },
    inplace=True
)

**HERE WE ARE SETTING QUANTITILE NUMBER TO EQUALLY DISTRIBUTE VALUE**

In [12]:
r = pd.qcut(rfm_dataset['recency'], q=5, labels=range(5, 0, -1))
f = pd.qcut(rfm_dataset['frequency'], q=5, labels=range(1, 6))
m = pd.qcut(rfm_dataset['monetary'], q=5, labels=range(1, 6))

In [30]:
rfm = rfm_dataset

rfm

,recency,frequency,monetary
_CustomerID,,,
1,9,152,335933.6115
10,15,158,435122.1870
11,6,178,487614.2415
12,3,210,616719.2550
13,4,171,441003.2795
14,5,157,381450.0280
15,4,142,441668.3550
16,3,135,402938.7705
17,6,175,534027.3860


In [31]:
df.dtypes

OrderNumber                 object
Sales Channel               object
WarehouseCode               object
ProcuredDate                object
OrderDate           datetime64[ns]
ShipDate                    object
DeliveryDate                object
CurrencyCode                object
_SalesTeamID                 int64
_CustomerID                 object
_StoreID                     int64
_ProductID                   int64
Order Quantity               int64
Discount Applied           float64
Unit Price                 float64
Unit Cost                  float64
Revenue                    float64
dtype: object

**HERE WE ARE IMPORTING KMEANS TO FURTHER BREAK DOWN OUR DATA IN CLUSTERS SO THAT WE CAN FURTHER DOWNSIZE THE DATA FOR BETTER VISUALIZATION**

In [32]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaled=scaler.fit_transform(rfm)

In [34]:
kmeans=KMeans(n_clusters=4)
kmeans.fit(scaled)
rfm['Clusters']=(kmeans.labels_)

In [35]:
rfm

,recency,frequency,monetary,Clusters
_CustomerID,,,,
1,9,152,335933.6115,1
10,15,158,435122.1870,2
11,6,178,487614.2415,0
12,3,210,616719.2550,3
13,4,171,441003.2795,0
14,5,157,381450.0280,1
15,4,142,441668.3550,1
16,3,135,402938.7705,1
17,6,175,534027.3860,0


**HERE WE ARE BREAKING DOWN OUR CLUSTERS INTO FOUR GROUPS FOR VISUALIZATION**

In [36]:
group=rfm.groupby(['Clusters'])['recency','frequency','monetary'].mean()
group

C:\Users\27817\AppData\Local\Temp\ipykernel_7992\3454363574.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group=rfm.groupby(['Clusters'])['recency','frequency','monetary'].mean()


,recency,frequency,monetary
Clusters,,,
0,5.250000,169.350000,467180.944975
1,5.913043,148.826087,377701.851370
2,18.500000,161.833333,446307.736500
3,3.000000,210.000000,616719.255000


In [42]:
group.to_excel('monetary.xlsx', index=False)

In [37]:
def func(row):
    if row['Clusters']==0:
        return 'Turtles'
    elif row['Clusters']==1:
        return 'Fishes'
    elif row['Clusters']==2:
        return 'Dolphins'
    else:
        return 'Whales'
    

In [38]:
rfm['Conditions']=rfm.apply(func,axis=1)

In [39]:
rfm

,recency,frequency,monetary,Clusters,Conditions
_CustomerID,,,,,
1,9,152,335933.6115,1,Fishes
10,15,158,435122.1870,2,Dolphins
11,6,178,487614.2415,0,Turtles
12,3,210,616719.2550,3,Whales
13,4,171,441003.2795,0,Turtles
14,5,157,381450.0280,1,Fishes
15,4,142,441668.3550,1,Fishes
16,3,135,402938.7705,1,Fishes
17,6,175,534027.3860,0,Turtles


In [40]:
result=rfm['Conditions'].value_counts()

In [41]:
result

Fishes      23
Turtles     20
Dolphins     6
Whales       1
Name: Conditions, dtype: int64

**THEN WE ARE SAVING OUR DATA IN A EXCEL FORMAT SO THAT WE CAN VISUALIZE IT**

In [43]:
result.to_excel('result.xlsx', index=False)